# [MYCODE] 모델 정의 
## 요구사항 정의
- RAG internet source를 https://spartacodingclub.kr/blog/all-in-challenge_winner 로 설정.
    - RAG에서 활용할 source로 위의 링크를 전달합니다.
    - 사이트가 달라졌기 때문에 이전 실습 코드와 다르게 load 해야 합니다. 어디를 어떻게 수정해야 할지 고민해보도록 합시다.
    - LLM은 GPT를 사용하시면 됩니다. 모델은 `gpt-4o-mini`로 설정하시면 됩니다.
- GPT에게 `“ALL-in 코딩 공모전 수상작들을 요약해줘.”`를 물은 뒤의 답변을 출력.

## 스펙정의 
- gpt-4o-mini
- 환경변수 `load_doenv`

[MYCODE] .env 파일 로드     
opeanAI 환경 설정을 .env 파일로 업로드 해서 안전하게 보관하였습니다

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import bs4
import re
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini")

[MYCODE] WebBaseLoader

- 스파르타코딩 블로그 챌리지 수상자에 해당하는 태그 크롤링

In [5]:
loader = WebBaseLoader(
    encoding="utf-8",
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(["h2", "p"])),  # h2, p 태그만 추출
)

docs = loader.load()

[MYCODE]
chunkSize 만큼 조각하고 400자만 만큼 겹치게 처리해서 질문에 잘 답변하도록합니다.
- chunk-size : 1000
- over-lab: 400

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=400)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

[MYCODE] 
	1.	벡터 스토어에서 Retriever 생성: 검색 가능하도록 설정.    
	2.	유저 메시지로 문서 검색: 입력 메시지를 기반으로 관련 문서를 검색.    
	3.	문서 형식화: 검색된 문서를 RAG 프롬프트에 맞는 형식으로 변환.    
	4.	RAG 프롬프트 로드 및 실행: 문서와 질문을 기반으로 요약된 답변 생성.    
	5.	결과 출력: 생성된 요약 내용을 출력.    

In [ ]:
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


prompt = hub.pull("rlm/rag-prompt")
print("prompt:::", prompt)

user_prompt = prompt.invoke(
    {"context": format_docs(retrieved_docs), "question": user_msg}
)
print(user_prompt)

prompt::: input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]
messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상

[MYCODE] 최종결과 

In [9]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전에서 수상한 주요 작품으로는 언어 학습을 돕는 웹 서비스 'Lexi Note', 벌레 퇴치 서비스 '우리집 히어로즈', 실시간 소통을 지원하는 '에코 클래스룸', 연합 동아리 정보 플랫폼 'Crewing', 매칭 및 교류 플랫폼 'BLOTIE' 등이 있습니다. 이들 프로젝트는 각각 대학생들이 겪는 다양한 문제를 해결하기 위한 창의적인 아이디어로 개발되었습니다. 각 작품은 독특한 기술 스택을 활용하여 실제 사용자에게 유용한 서비스를 제공하고 있습니다.
